In [1]:
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken
import time
from google.cloud import storage
import csv
import json
pd.set_option('display.max_colwidth', None)
time.sleep(1.0)
COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDING_MODEL = "text-embedding-ada-002"

In [2]:
#!gcsfuse generative-ai /home/jupyter/bucket

Files save in Buckets mlops_documents

In [3]:
from google.cloud import storage
client = storage.Client()
BUCKET_NAME = 'mlops_documents'
bucket = client.get_bucket(BUCKET_NAME)
elements = bucket.list_blobs()
files=[a.name for a in elements]

In [4]:
files

['df1.csv', 'df_clean.csv', 'df_clean_10_documents.csv', 'embedding.txt']

In [5]:
#pip install --upgrade google-cloud-storage

Import the file into Notebook from mlops_documents bucket

In [6]:
bucket_name = "mlops_documents"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

# When you have your files in a subfolder of the bucket.
my_prefix = "csv/" # the name of the subfolder
blobs = bucket.list_blobs(prefix = my_prefix, delimiter = '/')

for blob in blobs:
    if(blob.name != my_prefix): # ignoring the subfolder itself 
        file_name = blob.name.replace(my_prefix, "")
        blob.download_to_filename(file_name) # download the file to the machine
        df = pd.read_csv(file_name) # load the data
        print(df)

# When you have your files on the first level of your bucket

blobs = bucket.list_blobs()

for blob in blobs:
    file_name = blob.name
    blob.download_to_filename(file_name)
    data = pd.read_csv('df_clean_10_documents.csv') # load the data
data.head(1)

,filename,text content
0,AllInOnAI PREVIEW_10.txt,"What Does It Mean to Be AI Fueled?Some— but not nearly enough—of the most successful and most technological organizations in the world have declared their intention to be all-in on artificial intelligence, or “AI first” or “AI fueled.” Google described it as “a world that is AI- first, where computing becomes universally available—be it at home, at work, in the car, or on the go— and interacting with all of these surfaces becomes much more natural and intuitive, and above all, more intelligent.” Companies seeking to be powered by AI in other industries share the goal of intuitive technology and pervasive intelligence but are applying those objectives to their own industries, such as financial services, manufacturing, or health care.The AI- fueled organizations in our analysis comprise less than 1 percent of large companies. It wasn’t easy to find enough to write about in this book, but we were able to discover about thirty. We expect, however, that many more organizations will move in this direction. And why wouldn’t they? The companies we describe inthis book perform well. They have effective business models, make good decisions, have close relationships with customers, offer desir-able products and services, and charge profitable prices."


In [7]:
data.shape

(260, 2)

In [8]:
data['text content'] = data['text content'].replace(np.nan, '')

Tokenization

In [9]:
import nltk 
nltk.download('punkt')
data['tokenized_sents'] = data.apply(lambda row: nltk.word_tokenize(row['text content']), axis=1)

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
data['len_tokens'] = data['tokenized_sents'].apply(lambda x: len(x))

In [11]:
#data['tokenized_sents']

In [12]:
data['tokenized_text'] = [' '.join(map(str, l)) for l in data['tokenized_sents']]
#data['tokenized_text']

In [13]:
data = data[data['len_tokens']>40]
#data.head(1)

In [14]:
#reference https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb
prompt = "Who won the 2020 Summer Olympics men's high jump?"
openai.api_key = "yourkey"
openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

"Marcelo Chierighini of Brazil won the gold medal in the men's high jump at the 2020 Summer Olympics."

In [15]:
import time
#reference https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb
def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list:
    time.sleep(1.0)
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    return result["data"][0]["embedding"]
    
def compute_doc_embeddings(df: pd.DataFrame) -> list:
    time.sleep(1.0)
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.
    
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_embedding(r.content) for idx, r in df.iterrows()
    }

In [16]:
def load_embeddings(fname: str) -> dict:
    time.sleep(1.0)
    """
    Read the document embeddings and their keys from a CSV.
    
    fname is the path to a CSV with exactly these named columns: 
        "title", "heading", "0", "1", ... up to the length of the embedding vectors.
    """
    
    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df.columns if c != "title" and c != "heading"])
    return {
           (r.title, r.heading): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
    }

In [17]:
#data=data[['filename', 'text content', 'len_tokens','tokenized_text']]

In [18]:
data.rename(columns={'filename':'title','text content':'heading','tokenized_text':'content','len_tokens':'tokens'}, inplace=True)

In [19]:
data = data.set_index(["title", "heading"])

In [20]:
data.drop(['tokenized_sents'], axis=1,inplace=True)

In [21]:
data.columns

Index(['tokens', 'content'], dtype='object')

In [22]:
data=data[['content','tokens']]
#data.reset_index(inplace=True)

In [23]:
#data.head(1)

In [24]:
data.shape

(254, 2)

In [25]:
#data.to_csv('/data.csv')

In [26]:
#time.sleep(1.0)
#document_embeddings = compute_doc_embeddings(data)

In [27]:
import io
from google.cloud import storage

# Create a storage client
storage_client = storage.Client()

# Get the bucket
bucket = storage_client.get_bucket("mlops_documents")

# Get the blob
blob = bucket.get_blob("embedding.txt")

# Read the contents of the blob
blob_string = blob.download_as_string()

# Create a file-like object
blob_file = io.BytesIO(blob_string)

# Read the file
file_contents = blob_file.read()
#file_contents
# Print the contents of the file
#print(file_contents)

In [28]:
import ast
document_embeddings = ast.literal_eval(file_contents.decode('utf-8'))

In [29]:
#load_embeddings('my_dict.csv')

In [30]:
example_entry = list(document_embeddings.items())[0]
print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

('AllInOnAI PREVIEW_10.txt', 'What Does It Mean to Be AI Fueled?Some— but not nearly enough—of the most successful and most technological organizations in the world have declared their intention to be all-in on artificial intelligence, or “AI first” or “AI fueled.” Google described it as “a world that is AI- first, where computing becomes universally available—be it at home, at work, in the car, or on the go— and interacting with all of  these surfaces becomes much more natural and intuitive, and above all, more intelligent.” Companies seeking to be powered by AI in other industries share the goal of intuitive technology and pervasive intelligence but are applying those objectives to their own industries, such as financial services, manufacturing, or health care.The AI- fueled organizations in our analysis comprise less than 1\xa0percent of large companies. It wasn’t easy to find enough to write about in this book, but we  were able to discover about thirty. We expect, however, that ma

In [31]:
def vector_similarity(x: list, y: list) -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query: str, contexts: dict) -> list:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

In [32]:
order_document_sections_by_query_similarity("Benefits of Ml", document_embeddings)[:5]

[(0.8160402677862534,
  ('mlops-intelligent-products-essentials_6.txt',
   'ML engineersIntelligent Products Essentials helps ML engineers to automate the operation of ML models in a timely and reliable fashion. ML engineers manage the CI/CD pipelines that support the deployment of the ML pipeline, model, and in some cases, the prediction service.The following diagram shows the MLOps architecture of Intelligent Products Essentials from the perspective of ML engineers.')),
 (0.8119177672136959,
  ('MLOps Continuous delivery and automation pipelines in machine learning_2.txt',
   "DevOps versus MLOpsDevOps is a popular practice in developing and operating large-scale software systems. This practice provides benefits such as shortening the development cycles, increasing deployment velocity, and dependable releases. To achieve these benefits, you introduce two concepts in the software system development:Continuous Integration (CI)Continuous Delivery (CD)An ML system is a software system, s

In [33]:
order_document_sections_by_query_similarity("challenges of ML adoption", document_embeddings)[:5]

[(0.8944063934338102,
  ('Delivering on the Vision of MLOps_3.txt',
   ' Use Cases are Becoming Vast for ML Pioneers Across a Wide Variety of Areas These are still early days for ML, and success is not a given. Adoption faces several challenges: •  Senior management does not always see ML as strategic, and it can be difficult to measure and manage the value of ML projects. •  ML initiatives can work in isolation from each other, resulting in difficulties aligning workflows between ML and other teams. •  To be effective, ML training requires large quantities of high-quality data, which creates significant overheads across data access, preparation, and ongoing management. •  ML/data science work requires a large amount of trial and error, making it hard to plan  the time required to complete a project. In response, ML adoption requires a cultural shift and a technology environment with people, processes, and platforms operating in the responsive, agile way organizations are looking to op

In [34]:
MAX_SECTION_LEN = 500
SEPARATOR = "\n* "
ENCODING = "cl100k_base"  # encoding for text-embedding-ada-002

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

f"Context separator contains {separator_len} tokens"

'Context separator contains 3 tokens'

In [35]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.content.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"

In [36]:
prompt = construct_prompt(
    "What are MLOps Drivers",
    document_embeddings,
    data
)

print("===\n", prompt)

Selected 1 document sections:
('mlops-intelligent-products-essentials_2.txt', "MLOpsAs described in Hidden technical debt in ML systems, the ML code is only a small part of mature ML systems. In addition to the ML code and high-quality data, you need a way to put your ML processes into operation.MLOps is a practice which helps companies to build, deploy and put your ML system into operation in a rapid, repeatable, and reliable manner. MLOps is an application of DevOps principles to ML systems. MLOps is an engineering culture and practice that's intended to unify ML system development (Dev) and ML system operation (Ops). The objective of MLOps is to provide a set of standardized processes and technology capabilities for building, deploying, and putting ML systems into operation rapidly and reliably.The following sections discuss how MLOps can be implemented with Intelligent Products Essentials and Vertex AI.MLOps personasThe preceding diagram shows the following component and core MLOps

In [37]:
COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "max_tokens": 300,
    "model": COMPLETIONS_MODEL,
}

In [38]:
def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    document_embeddings: dict,
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

In [39]:
query = "can you provide multiple approaches to implement mlops"
answer = answer_query_with_context(query, data, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 1 document sections:
('Delivering on the Vision of MLOps_2.txt', 'Summary This report is targeted at Business and IT decision-makers as they look to implement MLOps, which is an approach to deliver Machine Learning- (ML-) based innovation projects. As well as describing how to address the impact of ML across the development cycle, it presents an approach based on maturity levels such that the organization can build on existing progress.The paper is for practitioners who require practical advice on MLOps, rather than general principles of ML. It references ML-related activities and their importance but does not go into technical detail about the specifics of ML nor associated activities.While the paper uses Azure Machine Learning and its documentation as a reference point, its guidance will apply to any ML environment.Drivers to MLOps Over the decades, data has proven to be a competitive differentiator. Once it was exclusively reports built by IT from overnight-batch-loaded dat

In [40]:
query = "Give me examples of ML use cases"
answer = answer_query_with_context(query, data, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 2 document sections:
('Delivering on the Vision of MLOps_3.txt', ' Use Cases are Becoming Vast for ML Pioneers Across a Wide Variety of Areas These are still early days for ML, and success is not a given. Adoption faces several challenges: •  Senior management does not always see ML as strategic, and it can be difficult to measure and manage the value of ML projects. •  ML initiatives can work in isolation from each other, resulting in difficulties aligning workflows between ML and other teams. •  To be effective, ML training requires large quantities of high-quality data, which creates significant overheads across data access, preparation, and ongoing management. •  ML/data science work requires a large amount of trial and error, making it hard to plan  the time required to complete a project. In response, ML adoption requires a cultural shift and a technology environment with people, processes, and platforms operating in the responsive, agile way organizations are looking to

In [41]:
query = "Johnson Controls has how many sensors on their chillers"
answer = answer_query_with_context(query, data, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 2 document sections:
('Delivering on the Vision of MLOps_29.txt', '  Because data science teams now have the ability to process the data, Johnson Controls now has up to 70 different types of sensors on their chillers with more coming online.  Overall results are that both unplanned downtime and mean time to repair are down by two-thirds. Chiller shutdowns, a highly detrimental event for JCI, has been drastically reduced. They predict potential shutdowns several days before and can comfortably intervene to keep customers happy and save money and time. “Using the MLOps capabilities in Azure Machine Learning, we were able to increase productivity and enhance operations, going to production in a timely fashion and creating a repeatable process,” says Chennupati.')
('Delivering on the Vision of MLOps_27.txt', ' Company Background  Microsoft (Nasdaq “MSFT” @microsoft) enables digital transformation for the era of an intelligent cloud and an intelligent edge. Its mission is to empowe

In [43]:
query = "how should I deploy ML models"
answer = answer_query_with_context(query, data, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 2 document sections:
('Google_User_15.txt', 'Ways of deploying your modelMicroservices: API-ify your model (Pickle, Joblib) Shared service: host your model in a dedicated tool, possibly automatedDeploy model together with your application  Streaming: load your model in memory (PMML, ONNX)(Python, MLlib)Deploy model as SQL stored procedureMany cloud providers and ML tools provide solutions for model deployment that integrate closely with their machine learning and data environments. These can greatly speed up deployment and ease infrastructure overhead such as:•  GCP Vertex AI•  AWS Sagemaker•  MLFlow')
('MLOps Continuous delivery and automation pipelines in machine learning_3.txt', "Deployment: In ML systems, deployment isn't as simple as deploying an offline-trained ML model as a prediction service. ML systems can require you to deploy a multi-step pipeline to automatically retrain and deploy model. This pipeline adds complexity and requires you to automate steps that are man

#### To store the embeddings txt format

In [87]:
# Create a bucket.
#bucket = storage_client.bucket('mlops_documents')

# Write the dictionary to a file.
#with open('embedding.txt', 'w') as f:
    #f.write(str(document_embeddings))

# Upload the file to the bucket.
#blob = bucket.blob('embedding.txt')
#blob.upload_from_filename('embedding.txt')

#### To store the embeddings csv format

In [88]:
# Create a gcloud storage client
#from google.cloud import storage
#import csv

#client = storage.Client()

# Create a gcloud bucket
#bucket = client.get_bucket('mlops_documents')

# Create a file in the bucket
#blob = bucket.blob('my_dict.csv')

# Create the csv file
#with open('my_dict.csv', 'w', newline='') as csvfile:
    # Create a csv writer
    #writer = csv.DictWriter(csvfile, fieldnames=document_embeddings.keys())

    # Write the csv file
    #writer.writeheader()
    #writer.writerow(document_embeddings)

# Upload csv file to the bucket
#blob.upload_from_filename('my_dict.csv')